# Setting up

Google Colab let us access a remote machine to run the protocol rather than our local computer. Each time we start a runtime session we need to install and configure the software to use.


## Installing miniconda

In [8]:
%%bash

wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -u

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local


## Setting up environment

There is no need to create a new environmnet as the session is reset every time we log out. However, we need to set the Python version and the packages to be used in the working conda base environment.

In [18]:
%%bash

conda install -c conda-forge python=3.9 numpy=1.21 pandas matplotlib pytest ambertools<<EOF
y
EOF

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ambertools
    - matplotlib
    - numpy=1.21
    - pandas
    - pytest
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    alsa-lib-1.2.13            |       hb9d3cd8_0         547 KB  conda-forge
    ambertools-23.3            |   py39h4856ba4_2        89.0 MB  conda-forge
    anaconda-anon-usage-0.5.0  | py39hfc0e8ea_100          27 KB
    arpack-3.7.0               |       hdefa2d7_2         215 KB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge
    blas-1.1                   |         openblas

Here, the `AMBERHOME` environment variable, which is required to run all amber-related modules, is not automatically set during `ambertools` installation. Set it manually with `os`.

In [33]:
import os

os.environ[ 'AMBERHOME' ] = '/usr/local'

## Installing GROMACS and FATSLiM

In [36]:
!sudo apt install gromacs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-mathjax gromacs-data libfftw3-double3 libfftw3-single3 libgromacs6
  libjs-mathjax sse4.2-support
Suggested packages:
  pymol libfftw3-bin libfftw3-dev fonts-mathjax-extras fonts-stix
  libjs-mathjax-doc
The following NEW packages will be installed:
  fonts-mathjax gromacs gromacs-data libfftw3-double3 libfftw3-single3
  libgromacs6 libjs-mathjax sse4.2-support
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 58.3 MB of archives.
After this operation, 310 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 sse4.2-support amd64 6 [8,572 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-mathjax all 2.7.9+dfsg-1 [2,208 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubuntu8 [770 kB]
Get:4 http://archiv

Then install FATSLiM. Additionally, test that there are no major errors with the self-test functionality.

In [19]:
!pip install fatslim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 100.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fatslim: filename=fatslim-0.2.2-cp39-cp39-linux_x86_64.whl size=22268075 sha256=219bed651b268ae95550de23b71b061ab2d1d129884d49671989e30a2f465508
  Stored in directory: /root/.cache/pip/wheels/73/53/b4/17fd60698c5430719ffa2bbcb511b824ceff6ed7ace1bebdc2
Successfully built fatslim


In [20]:
!fatslim self-test

FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

FATSLiM version: 0.2.2
Python version: 3.9.21 (/usr/local/bin/python3.9)
Cython version (file generation): 0.29.14
Python compiler: GCC 13.3.0
CPU architecture: 64bit
OpenMP: 2 CPUs (default number of threads: 2 - max: 2)
NumPy version: 1.21.6
======================================= test session starts ========================================
platform linux -- Python 3.9.21, pytest-8.3.4, pluggy-1.5.0
rootdir: /usr/local/lib/python3.9/site-packages/fatslimlib/test
collected 196 items                                                                                

../usr/local/lib/python3.9/site-packages/fatslimlib/test/test_00_cosmogony.py .............. [  7%]
......                                                                                       [ 10%]
../usr/local/lib/python3.9/site-packages/fatslimlib/test/test_01_datareading.py ............ [ 16%]
......

# Protocol

Before you start, clone this repo anywhere you want in your local machine. Here in Colab, you automatically start in the `/content/` directory.

In [21]:
!git clone https://github.com/alquin97/md_membrane_class.git

Cloning into 'md_membrane_class'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 84 (delta 24), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 15.26 MiB | 14.82 MiB/s, done.
Resolving deltas: 100% (24/24), done.


## Simple membrane system

### Just POPC

This part of the protocol will be done under `just_popc/`:

In [22]:
%cd /content/md_membrane_class/practical/just_popc

/content/md_membrane_class/practical/just_popc


#### Membrane creation with PACKMOL-Memgen

First, create a membrane with just POPC using [PACKMOL-Memgen](https://pubs.acs.org/doi/10.1021/acs.jcim.9b00269). With the following command, we're defining a 75x75 A membrane.

In [35]:
!packmol-memgen --lipids POPC --distxy_fix 75

PMEMD not found. Setting path to SANDER


 _____           _                    _
|  __ \         | |                  | |
| |__) |_ _  ___| | ___ __ ___   ___ | |
|  ___/ _` |/ __| |/ / '_ ` _ \ / _ \| |
| |  | (_| | (__|   <| | | | | | (_) | |
|_|   \__,_|\___|_|\_\_| |_| |_|\___/|_|
                                         ___
                  /\/\   ___ _ __ ___   / _ \___ _ __
                 /    \ / _ \ '_ ` _ \ / /_\/ _ \ '_ \ 
                / /\/\ \  __/ | | | | / /_ \  __/ | | |
                \/    \/\___|_| |_| |_\____/\___|_| |_|


###############################################################
Stephan Schott-Verdugo 2016-11-07        VERSION: 2023.2.24
Generated at CPCLab at Heinrich Heine University Duesseldorf
      &      CBCLab at Forschungszentrum Juelich
###############################################################

--verbose for details of the packing process
No ratio provided (--ratio).

 Information for packing:

 Input PDB(s)                     = None   

> Note: We are not adding any concentration of ions. Ideally there should be a salt concentration of 0.15M to replicate more accurately the real conditions. However, for the purposes of the tutorial, we're not adding any. Our system is already neutral so it won't necessarily affect the electrostatics.

This process will take some minutes. Once it's done, you can check the resulting membrane with VMD or other visualization software such as [PyMOL](https://www.pymol.org) in your local machine.

#### System topology and transformation to GROMACS

Next, we need to obtain to obtain the Amber force field parameters (version 14SB, ff14SB) for our system. This is done with the processing tool `leap` that will output a .prmtop and .inpcrd files provided of a PDB file.

In [38]:
!cp ../files/leap.in .

In [39]:
!tleap -f leap.in

-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./leap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/oldff/leaprc.lipid14
----- Source of /usr/local/dat/leap/cm

Recent versions of PACKMOL-Memgen tend to produce errors in GROMACS due to structure clashing. Perform a small minimization of the system with the `sander` module to correct it. Input parameters for `sander` are stated in a `system.sander` file. This process can take some time, you can check the progress in the  `system.sanderout` file.

In [41]:
!cp ../files/system.sander .

In [62]:
!sander -O -i system.sander -o system.sanderout -p system.prmtop -c system.inpcrd -r system.rst -ref system.inpcrd


  Error opening unit    6: File "system.sanderout" exists (use -O to overwrite)            


At the moment, we have the required parameters to run the system in [Amber](https://ambermd.org/AmberMD.php) (equally valid). However, we are simulating in [GROMACS](https://manual.gromacs.org/), so we need to transform them to a .top and .gro files. This is done using `amb2gro_top_gro.py`. Additionally, output the minimization end coordinates with the `-b` option.

In [43]:
!amb2gro_top_gro.py -p system.prmtop -c system.rst -t system_GMX.top -g system_GMX.gro -b system_out.pdb

Finally, copy the 'molecular dynamics parameters' files (.mdp) required for the following parts.

In [44]:
!cp -r ../files/mdp .

#### Energy minimization

We are going to continue with a short minimization (1000 steps), now in GROMACS. Open the file `mdp/min.mdp` in a text editor to check what you are going to do first.

Execute `gmx grompp` to generate a portable binary run file (.tpr), which contains the starting structure of your simulation, the molecular topology and all the simulation parameters.


In [45]:
!gmx grompp -f mdp/min.mdp -r system_GMX.gro -c system_GMX.gro -p system_GMX.top -o system_min.tpr -maxwarn 1

              :-) GROMACS - gmx grompp, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

> Note: Read carefully the NOTES and WARNINGS that are printed during grompp. Usually these are a dead giveaway of wrongdoings that doom your runs to failure.

Then, run `gmx mdrun` to perform the actual calculation.

In [46]:
!gmx mdrun -deffnm system_min -v

              :-) GROMACS - gmx mdrun, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            T

#### Equilibration

We are going to perform a three-step equilibration of our system on a NPT ensemble. For this, we need to add a positional restraints statement in the `system_GMX.top` for the lipids which will be gradually tuned down to reach equilibrium. This is done by including the following lines within the lipids' [ moleculetype ] section (named 'system1') but before the water's [ moleculetype ] section begins (named WAT, which can have its own set of positional restraints).

```
#ifdef POSRES
#include "posre.itp"
#endif
```

Thus the `system_GMX.top` file should look something like this.

```
[ moleculetype ]
; Name            nrexcl
system1          3

...

#ifdef POSRES
#include "posre.itp"
#endif

[ moleculetype ]
; Name            nrexcl
WAT          3

...
```
**IMPORTANT: Edit the system_GMX.top locally with a text editor or use edit the file with python in a new cell.**

The sequential equilibration runs will be executed through the `equi.sh` shell script. Examine the script, it contains four different sections. Figure out what each of them does. Notice also that each equilibration step has its own `equi*.mdp`.

> Hint: you can use `vimdiff file1 file2` to compare two files and spot the differences more easily.

Once you feel ready, execute the `equi.sh` shell script.


In [60]:
!chmod +x equi.sh
!bash equi.sh

             :-) GROMACS - gmx make_ndx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff           

**IMPORTANT: Without GPU support, the whole equilibration will take a long while. We are not gonna wait for all the steps to finish. Kill the process with `CTRL-C` and copy the `just_popc/*` files from the shared folder provided at the beginning of the class. Then continue with the protocol.**



In [67]:
!cp /content/shared_files/just_popc/* .

To visualize the time-evolution of the trajectory of the first equilibration step (equi1) use VMD (or any other visualization software).

> Note: The .trr file in GROMACS contains the actual trajectory, but it's best to visualize it using the compressed file .xtc

We are also going to assess some of the variables of this equilibration step. We can do that using GROMACS' analysis tools such as `gmx energy`. Pick the variables we want to assess by typing the following numbers.

`12 14 15 20 21 0`



In [69]:
%%bash

gmx energy -f system_equi1.edr -o equi1.xvg<<EOF
12 14 15 20 21 0
EOF


Statistics over 100001 steps [ 0.0000 through 100.0000 ps ], 5 data sets
All statistics are over 1001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Total Energy                -203067       3600    8615.96   -22742.8  (kJ/mol)
Temperature                 299.774       0.13    2.64375  -0.105134  (K)
Pressure                   -65.5879         31    192.691    203.668  (bar)
Volume                      460.104         22    45.4145   -141.245  (nm^3)
Density                     919.038         39    81.6608     262.37  (kg/m^3)


              :-) GROMACS - gmx energy, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

This way we are selecting the **total energy of the system**, **temperature**, **pressure**, **density** and **volume**. The final zero exits the prompt. To visualize the file you can either use `xmgrace` or execute the python script provided in this repo.

In [77]:
!python ../files/plot_xvg.py equi1.xvg

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 26, in <module>
    from pandas.compat import (
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py", line 26, in <module>
    from pandas.compat.numpy import is_numpy_dev
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py", line 21, in <module>
    raise ImportError(
ImportError: this version of pandas is incompatible with numpy < 1.22.4
your numpy version is 1.21.6.
Please upgrade numpy to >= 1.22.4 to use this pandas version

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/md_membrane_class/practical/just_popc/../files/plot_xvg.py", line 2, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 31, in <module>
    raise ImportError(
ImportError: C extension: None not built. If you want to import pandas 

It outputs a .png image in the same location where the .xvg file is located. See how the different variables change along time until stabilized.

#### Production

We are ready to start producing the proper simulation. We are going to do a 100 ns long unbiased MD simulation. Again, we execute `gmx grompp` and then `gmx mdrun`.

In [80]:
!gmx grompp -f mdp/prod.mdp -r system_equi3.gro -c system_equi3.gro -p system_GMX.top -o system_prod.tpr -n index.ndx -maxwarn 1

              :-) GROMACS - gmx grompp, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

> Note: See what has changed between the equi3.mdp and prod.mdp files.

In [81]:
!gmx mdrun -deffnm system_prod -v

              :-) GROMACS - gmx mdrun, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            T

**IMPORTANT: Two problems may arise here:**
- **1) `gmx grompp` doesn't go through because system_equi3.gro file and system_GMX.top file don't match in atom number (inescapable condition). If you got the shared system_equi3.gro this might be true as the two systems were built slightly different (different PACKMOL-Memgen runs).**
- **2) Again, the simulation run takes too long to finish. Proceed with the shared output files.**

Once finished, see what the simulation looks like.

#### Analysis

Next, we will do a short analysis of our membrane. The two typical measurements to examine are the **membrane thickness** and the **area per lipid (APL)**. Use [FATSLiM](http://fatslim.github.io/) to analyze membrane simulations. FATSLiM makes use of lipid 'head groups' to calculate these two values, which must be defined in a index file. Use GROMACS to create a new index file with the lipid head groups properly indexed.

In [83]:
%%bash

gmx make_ndx -f system_equi3.gro -o fatslim.ndx<<EOF
a P31
name 8 headgroups
q
EOF

Going to read 0 old index file(s)
Analysing residue names:
There are:   504      Other residues
There are:  6566      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 42210 atoms
  1 Other               : 22512 atoms
  2 PA                  :  7728 atoms
  3 PC                  :  6384 atoms
  4 OL                  :  8400 atoms
  5 Water               : 19698 atoms
  6 SOL                 : 19698 atoms
  7 non-Water           : 22512 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "name": group             'case': case sensitive           'q': save and quit
 'ri': residue index

> 
Found 168 atoms with name P31

  8 P31                 :   168 atoms

> 

> 


             :-) GROMACS - gmx make_ndx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff           

> Question: Which atom is the head group of POPC?

##### Membrane thickness

To determine the membrane thickness run the following command.

In [84]:
!fatslim thickness -c system_equi3.gro -t system_prod.xtc -n fatslim.ndx --plot-thickness thickness.xvg

FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'thickness'... This may take some time, be patient!
Analysis will be performed using 2 threads.
Analysing frame  1001/ 1001 (time: 100000 ps)... done in 16 ms (Remaining: 0 s)
Results:
Average values over 1001 processed frames:
Thickness: Membrane=3.885±0.064nm - Lower leaflet=3.888±0.065nm - Upper leaflet=3.882±0.066nm
Thickness values saved to 'thickness.xvg'


DEBUG!
'thickness' command executed in 21.570 s (CPU)
Goodbye!


The software will give us both the thickness per leaflet and for the whole membrane. Moreover, with the option `--plot-thickness` we can obtain a plot of the thickness over time. You can again use `xmgrace` or the provided python script to plot the .xvg file.

In [85]:
!python ../files/plot_xvg.py thickness.xvg

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 26, in <module>
    from pandas.compat import (
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py", line 26, in <module>
    from pandas.compat.numpy import is_numpy_dev
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py", line 21, in <module>
    raise ImportError(
ImportError: this version of pandas is incompatible with numpy < 1.22.4
your numpy version is 1.21.6.
Please upgrade numpy to >= 1.22.4 to use this pandas version

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/md_membrane_class/practical/just_popc/../files/plot_xvg.py", line 2, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 31, in <module>
    raise ImportError(
ImportError: C extension: None not built. If you want to import pandas 

##### Area per lipid (APL)

The APL is calculated with the following command.

In [86]:
!fatslim apl -c system_equi3.gro -t system_prod.xtc -n fatslim.ndx --plot-apl apl.xvg

FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 2 threads.
Analysing frame  1001/ 1001 (time: 100000 ps)... done in 16 ms (Remaining: 0 s)
Results:
Average values over 1001 processed frames:
Area per lipid: Membrane=0.676±0.014nm^2 - Lower leaflet=0.646±0.013nm^2 - Upper leaflet=0.710±0.015nm^2
Area per lipid values saved to 'apl.xvg'

Area: Membrane=56.728±1.137nm^2 - Lower leaflet=56.793±1.140nm^2 - Upper leaflet=56.664±1.157nm^2

DEBUG!
'apl' command executed in 21.843 s (CPU)
Goodbye!



And, just like before, the APL per leaflet, for the whole membrane, and a plot over time is generated.

In [87]:
!python ../files/plot_xvg.py apl.xvg

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 26, in <module>
    from pandas.compat import (
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py", line 26, in <module>
    from pandas.compat.numpy import is_numpy_dev
  File "/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py", line 21, in <module>
    raise ImportError(
ImportError: this version of pandas is incompatible with numpy < 1.22.4
your numpy version is 1.21.6.
Please upgrade numpy to >= 1.22.4 to use this pandas version

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/md_membrane_class/practical/just_popc/../files/plot_xvg.py", line 2, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.9/site-packages/pandas/__init__.py", line 31, in <module>
    raise ImportError(
ImportError: C extension: None not built. If you want to import pandas 

### POPC+CHL

This part of the protocol will be done under `popc+chl/`:

In [100]:
%cd /content/md_membrane_class/practical/popc+chl/

/content/md_membrane_class/practical/popc+chl


In this case, we are going to simulate a membrane with a ratio of 1:3 cholesterol molecules per POPC. Create the membrane with PACKMOL-Memgen with the apropiate input flags.


In [89]:
!packmol-memgen --lipids POPC:CHL1 --ratio 3:1 --distxy_fix 75

PMEMD not found. Setting path to SANDER


 _____           _                    _
|  __ \         | |                  | |
| |__) |_ _  ___| | ___ __ ___   ___ | |
|  ___/ _` |/ __| |/ / '_ ` _ \ / _ \| |
| |  | (_| | (__|   <| | | | | | (_) | |
|_|   \__,_|\___|_|\_\_| |_| |_|\___/|_|
                                         ___
                  /\/\   ___ _ __ ___   / _ \___ _ __
                 /    \ / _ \ '_ ` _ \ / /_\/ _ \ '_ \ 
                / /\/\ \  __/ | | | | / /_ \  __/ | | |
                \/    \/\___|_| |_| |_\____/\___|_| |_|


###############################################################
Stephan Schott-Verdugo 2016-11-07        VERSION: 2023.2.24
Generated at CPCLab at Heinrich Heine University Duesseldorf
      &      CBCLab at Forschungszentrum Juelich
###############################################################

--verbose for details of the packing process

 Information for packing:

 Input PDB(s)                     = None     
 Output PDB               

Visualize the system (in VMD or PyMOL) and spot the cholesterol molecules.

> Note: We are gonna skip the system preparation that we did before for the just POPC membrane. If you attempt to run the same preparation protocol for the POPC+CHL membrane, consider that there must be two instances of lipid [ moleculetype ] (one for POPC and one for CHL) in the GROMACS .top file, each with their corresponding positional restraints for the equilibration.

In the same shared folder provided before you will find a 100 ns simulation of a similar POPC+CHL membrane system. Repeat the previous analysis to measure the membrane thickness and APL, and compare those to the POPC-only membrane.

> Hint: Now there are two molecule types contributing to the head groups.

In [101]:
!cp /content/shared_files/popc+chl/* .

In [102]:
%%bash

gmx make_ndx -f system_equi3.gro -o fatslim.ndx<<EOF
r PC | r CHL & a P31 | a O1
name 9 headgroups
q
EOF

Going to read 0 old index file(s)
Analysing residue names:
There are:   486      Other residues
There are:  6586      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 42874 atoms
  1 Other               : 23116 atoms
  2 PA                  :  6716 atoms
  3 PC                  :  5548 atoms
  4 OL                  :  7300 atoms
  5 CHL                 :  3552 atoms
  6 Water               : 19758 atoms
  7 SOL                 : 19758 atoms
  8 non-Water           : 23116 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "name": group             'case': case sensitive           'q': save and quit
 'ri': residue index

> 
Found 5548 atoms with residue name PC
Found 35

             :-) GROMACS - gmx make_ndx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff           

> Question: Which atom is the head group of CHL?

## Protein-Membrane system

This part of the protocol will be done under `membrane_protein/`:

In [103]:
%cd /content/md_membrane_class/practical/membrane_protein/

/content/md_membrane_class/practical/membrane_protein


### Building the system

This is the real deal. We are going to embed a membrane protein in a lipid bilayer. Our membrane protein is a refined structure of the Cannabinoid Receptor 2 (CB2), a class A GPCR that contains 7 characteristic transmembrane helices.

Run PACKMOL-Memgen on the provided `protein.pdb` file. Then visualize the system (with VMD or PyMOL).


In [ ]:
!packmol-memgen --pdb protein.pdb --lipids POPC:CHL1 --ratio 10:1 \
    --dist 12 --dist_wat 15 --salt --salt_c Na+

> Note: Here, the membrane is built 12 A from the protein to the edges of the box in the X and Y axis and 15 A in the Z axis. We have chosen a concentration of salt of 0.15 M (cations and anions can be specified).

Again, we are not gonna go over the preparation steps we followed before (because it's gonna take longer than the previous parts). So we are jumping straight to the analysis.



### Analysis

Copy the 100 ns trajectory of the CB2 receptor embedded in a 10:1 POPC:CHL membrane provided in the shared folder. In this section, we will focus our attention to values concerning the protein rather than the lipids.


In [98]:
!cp /content/shared_files/membrane_protein/* .

#### RMSD

First, measure the RMSD of the C-alpha atoms of the receptor along the trajectory. This can be easliy done with GROMACS. GROMACS will prompt us 1) which group to align the system coordinates to and 2) which group to compute the RMSD for. Select the `C-alpha` group in both cases (type `3` and press `Enter`). This way GROMACS will automatically align all the coordinates and calculate the RMSD for the C-alpha atoms of our protein. Once done, plot the .xvg file.


In [105]:
%%bash

gmx rms -f system_prod.xtc -s system_equi6.gro -o rmsd.xvg<<EOF
3
3
EOF


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

Selected 3: 'C-alpha'
Selected 3: 'C-alpha'


               :-) GROMACS - gmx rms, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Te

In [ ]:
!python ../files/plot_xvg.py rmsd.xvg

#### RMSF

Now, calculate the RMSF of the structure throughout the simulation. This will help us determine how stable our transmembrane alpha-helices are or whether there is some specially labile part of the protein. Select `C-alpha` as well. Plot the results.


In [ ]:
!python ../files/plot_xvg.py rmsf.xvg

#### Secondary Structure analysis

Last, we are going to perform a simple secondary structure analysis to further assess the stability of the transmembrane helices.


In [ ]:
!gmx dssp -f system_prod.xtc -s system_equi6.gro -o dssp.dat

GROMACS knows, based on the `index.ndx`, what the atoms of the Protein are. The output `dssp.dat` contains the secondary structure prediction per residue (each character) per frame (each line). Each letter stands for:
- H = α-helix
- B = residue in isolated β-bridge
- E = extended strand, participates in β ladder
- G = 3-helix (310 helix)
- I = 5 helix (π-helix)
- T = hydrogen bonded turn
- S = bend
- ~ = unknown

Plot the output file. You can use this trashy script.


In [ ]:
!python ../files/plot_dssp.py dssp.dat

> Note: This will return a `dssp.png` with a heatmap, where each color represents a letter (secondary structure), the sequence in the X-axis and each frame in the Y-axis.